In [1]:
import sagemaker
from sagemaker.local import LocalSession
import boto3
import json

session = boto3.Session(profile_name='dev-profile')
sagemaker_session = sagemaker.Session(boto_session=session)

iam = session.client('iam')
role = iam.get_role(RoleName='sagemaker-execution-role')['Role']['Arn']

print(f"sagemaker role arn: {role}")


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/katie/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::051285996764:role/sagemaker-execution-role


In [2]:
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'stabilityai/stablelm-zephyr-3b', # model_id from hf.co/models
  'HF_TASK':'question-answering' # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.37", # transformers version used
   pytorch_version="2.1", # pytorch version used
   py_version="py310", # python version of the DLC
)

In [3]:
# deploy model to SageMaker Inference

# TODO figure out why (InvalidClientTokenId) and how it might be linked to S3 bucket creation.
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

ClientError: An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.

In [ ]:
# example request, you always need to define "inputs"
data = {
"inputs": {
    "question": "What is used for inference?",
    "context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
    }
}

# request
predictor.predict(data)

In [ ]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()

In [4]:
session = boto3.Session(profile_name='dev-profile')
client = boto3.client("sagemaker-runtime")
 
# You can also get the endpoint name by inspecting the output of Terraform with terraform output 
# or going to the SageMaker service in the AWS Management console. 
ENDPOINT_NAME = "stablelm-zephyr-3b-endpoint"
 
body={"inputs":"This terraform module is amazing."}
 
response = client.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType="application/json",
            Accept="application/json",
            Body=json.dumps(body),
        )
print(response['Body'].read().decode('utf-8'))

ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeEndpoint operation: The security token included in the request is invalid.